In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

In [3]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import pandas as pd
np.random.seed(1)

### Helper functions for Data Loading, normalization, etc

In [ ]:
def Reshape_to_input(X, T, Mat_dim):
    """
    reshapes data
    """
   
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, Mat_dim, T)
    
    return input_X
    

In [ ]:
def NormalizeTowardsMean(X):
   # print(X.shape)
    "# Preprocessing: Subtract the mean feature - a question if it is feasible - all features are already in the same scale...\n",
    # TODO: try -1 1 normalization
    mean_feat = np.mean(X, axis=0, keepdims=True)
   # print(mean_feat.shape)
    X -= mean_feat
    # Preprocessing: Divide by standard deviation. This ensures that each feature\n",
    # has roughly the same scale.\n",
    std_feat = np.std(X, axis=0, keepdims=True)
    X /= std_feat
    
    return X



In [ ]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    Persons_train = np.array([])
    Persons_test = np.array([])
    #print(X_train.shape)
    for Person in range(0,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, returned_train
            
            


In [ ]:
def Random_Selection_train_val_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    X_val = np.array([])
    Y_val = np.array([])
    Persons_train = np.array([])
    Persons_val = np.array([])
    Persons_test = np.array([])
    
    train_N =int(np.ceil(N*2/3))
    
    for Person in range(0,train_N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    
     
    for Person in range(train_N,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_val= np.append(X_val,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_val = np.append(Y_val, X_all[[indexes], 0])
        Persons_val = np.append(Persons_val, label, axis = 0)
    X_val=X_val.reshape(Y_val.size, num_dim)  
    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for val: ')
    print(np.unique(Persons_val))
    returned_val = np.unique(Persons_val)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val
            
            


In [ ]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [ ]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units = 128 
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = Bidirectional(LSTM(units, return_sequences=True))( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(units, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
# load my dataset and parameters from a file
from data_utils import loadfromfolder

### Model Training

In [ ]:
myoptim=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')
    return [es, mcp_save, reduce_lr_loss]

In [ ]:
# parameters needed
trials = 3
epochs = 25
batch_size = 30
units = 192 # actually defind inside the funciton of the model - TODO - make it here
path ='data/2_angles_only'
Files, T, max_dim = loadfromfolder()
#Where Files are names, T - my division into frames and max_dim - feature dimensionality
fRes = open("Results.txt","a+") 
fRes.write("Number of units in LSTM = %d,  Epochs =  %d and batch size =  %d \r\n" % (units, epochs,batch_size))
for file in range(0, len(Files)):
    name = Files[file]
    t = T[file]
    dim = max_dim[file]
    name_w_path = path +"/"+name
    fRes.write("Experiment %d from file %s \r\n" % (file,name))
    # here all the routine in One cell 
    X_all = pd.read_csv(name_w_path)
    # the routine to run the same test N times, randomly shuffling the data
    Accuracy  = np.zeros(shape=(trials))
   
    
    for i in range(0,trials):
        X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val = Random_Selection_train_val_test (X_all.values, 14)
        #namePersons = str(i)+'_'+'file'+ name[:-4] +'.txt'
        #np.savetxt(namePersons, returned_train, delimiter=',', fmt='%d')   # X is an array
        X_train = NormalizeTowardsMean(X_train)
        X_test = NormalizeTowardsMean(X_test)
        X_val = NormalizeTowardsMean(X_val)
        
        Y_train_oh = convert_to_one_hot(Y_train, C = 3)
        Y_test_oh = convert_to_one_hot(Y_test, C = 3)
        Y_val_oh = convert_to_one_hot(Y_val, C = 3)
        
        X_train = Reshape_to_input(X_train, t, dim)
        X_test = Reshape_to_input(X_test, t, dim)
        X_val = Reshape_to_input(X_val, t, dim)
        
        model = Gait_model((X_train[0,:,:]))
        
        file_path = 'Models/file_{}_model_wts.hdf5'.format(file)
        callbacks = get_callbacks(filepath=file_path, patience = 35)
        model.compile(loss='categorical_crossentropy', optimizer=myoptim, metrics=['accuracy'])
              
        
        model.fit(X_train, Y_train_oh, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_val, Y_val_oh))
        
        loss, acc = model.evaluate(X_test, Y_test_oh)
        
        Accuracy[i]=acc
        fRes.write("Final accuracy per trial %f \r\n" % (acc))
        
    print(Accuracy)
    fRes.write("Final mean accuracy is: %f \r\n" % (np.mean(Accuracy)))
fRes.close() 


In [ ]:
fRes.close() 